In [ ]:
!pip install openai pandas datasets


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [89]:
import openai
from datasets import load_dataset
import pandas as pd
import re

In [90]:
import getpass

# Get your API key securely
api_key = getpass.getpass("Enter your OpenAI API key: ")

client = openai.OpenAI(api_key=api_key)  # Create a client instance


Enter your OpenAI API key:  ········


In [92]:
from datasets import load_dataset

# Load pubmedqa dataset (we'll use the 'pqa_labeled' subset)
dataset = load_dataset("pubmed_qa", "pqa_labeled")

# Select a smaller test subset for demonstration
examples = dataset['train'].select(range(10))  # You can increase this later


In [ ]:
# matched_rows = df[df['QUESTION'].str.contains("Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?", case=False, na=False)]


In [95]:
import pandas as pd

# Convert to DataFrame for easier viewing
df = pd.DataFrame({
    'PUB_ID': examples['pubid'],
    'QUESTION': examples['question'],
    'CONTEXT': examples['context'],
    'ANSWER': examples['final_decision'],  # Ground truth: 'yes', 'no', or 'maybe'
    'LONG_ANSWER': examples['long_answer']
})

with pd.option_context('display.max_colwidth', None):
    print(df.loc[1])


PUB_ID                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [97]:
def ask_gpt(question, context):
    prompt = (
        f"You are a biomedical expert assistant. Given the following abstract:\n\n"
        f"{context}\n\n"
        f"Question: {question}\n\n"
        f"Please answer with 'yes', 'no', or 'maybe' and explain your reasoning.\n\n"
        f"Answer:"
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=120,
        n=1,
    presence_penalty=0.0
    )

    return response.choices[0].message.content


In [99]:
def extract_answer(gpt_output):
    match = re.search(r"\b(yes|no|maybe)\b", gpt_output.lower())
    return match.group(1) if match else "unknown"

In [101]:
results = []

for i, row in df.iterrows():
    print(f"Processing question {i+1}...")
    gpt_response = ask_gpt(row['QUESTION'], row['CONTEXT'])
    predicted = extract_answer(gpt_response)

    results.append({
        'question': row['QUESTION'],
        'ground_truth': row['ANSWER'],
        'predicted_answer': predicted,
        'gpt_explanation': gpt_response,
         'reference_explanation': row['CONTEXT']  # <-- add this line
    })


Processing question 1...
Processing question 2...
Processing question 3...
Processing question 4...
Processing question 5...
Processing question 6...
Processing question 7...
Processing question 8...
Processing question 9...
Processing question 10...


In [102]:
data_point = results[0]

print("Question:", data_point['question'])
print("Ground Truth Answer:", data_point['ground_truth'])
print("Predicted Answer:", data_point['predicted_answer'])
print("GPT Explanation:", data_point['gpt_explanation'])
print("Reference Explanation (Abstract):", data_point['reference_explanation'])


Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
Ground Truth Answer: yes
Predicted Answer: yes
GPT Explanation: Yes, the abstract suggests that mitochondria play a role in the remodeling of lace plant leaves during programmed cell death. The study found that mitochondrial dynamics, including distribution, motility, and membrane potential, change during different stages of programmed cell death in the lace plant. Furthermore, treatment with cyclosporine A, which indirectly affects mitochondrial permeability transition pore formation, resulted in lace plant leaves with significantly fewer perforations, suggesting that mitochondria play a role in this process.
Reference Explanation (Abstract): {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longit

In [105]:
results_df = pd.DataFrame(results)
results_df.to_csv("gpt4_pubmedqa_results_05192025.csv", index=False)

accuracy = (results_df['ground_truth'].str.lower() == results_df['predicted_answer']).mean()
print(f"\nAccuracy on {len(results_df)} examples: {accuracy*100:.2f}%")
print("Saved results to gpt4_pubmedqa_results_05192025.csv")


Accuracy on 10 examples: 50.00%
Saved results to gpt4_pubmedqa_results_05192025.csv


In [107]:
#EXACT MATCH

correct = 0
total = len(results)

for r in results:
    if r['predicted_answer'].strip().lower() == r['ground_truth'].strip().lower():
        correct += 1

accuracy = correct / total * 100
print(f"Exact Match Accuracy: {accuracy:.2f}%")



Exact Match Accuracy: 50.00%


In [ ]:
!pip install bert-score


In [ ]:
from bert_score import score

# Extract lists of explanations
gpt_explanations = [r['gpt_explanation'] for r in results]
reference_explanations = [r['reference_explanation'] for r in results]

P, R, F1 = score(gpt_explanations, reference_explanations, lang="en", model_type="bert-base-uncased")
print(f"BERTScore F1: {F1.mean().item():.4f}")


In [ ]:
from bert_score import score

# Extract lists of explanations
gpt_explanations = [r['gpt_explanation'] for r in results]
reference_explanations = [r['reference_explanation'] for r in results]

P, R, F1 = score(gpt_explanations, reference_explanations, lang="en", model_type="bert-base-uncased")
print(f"BERTScore F1: {F1.mean().item():.4f}")


In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from bert_score import score as bert_score
# from bart_score import BARTScorer

# Extract lists of predictions and references
predictions = [r['gpt_explanation'] for r in results]
references = [r['reference_explanation'] for r in results]  # You need to have this in your data

# -------- BERTScore --------
# P, R, F1 = bert_score(predictions, references, lang="en", model_type="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", verbose=True)
from bert_score import score

P, R, F1 = score(
    predictions,
    references,
    lang="en",  # keep lang as 'en'
    model_type="bert-base-uncased",  # Use this instead
    verbose=True
)

# Average BERTScore F1
bert_f1_avg = F1.mean().item()
print(f"Average BERTScore F1: {bert_f1_avg:.4f}")

# -------- BARTScore --------
# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu', checkpoint='facebook/bart-large-cnn')

# Compute scores
# bart_scores = bart_scorer.score(predictions, references, batch_size=4)
# bart_avg = sum(bart_scores) / len(bart_scores)
# print(f"Average BARTScore: {bart_avg:.4f}")


In [ ]:
!git clone https://github.com/neulab/BARTScore.git


In [67]:
cd BARTScore

C:\Users\mnpri\ACP_GenAI_June\BARTScore


In [ ]:
predictions = [str(r['gpt_explanation']) for r in results if r['gpt_explanation'] is not None]
references = [str(r['reference_explanation']) for r in results if r['reference_explanation'] is not None]


In [ ]:
from bart_score import BARTScorer
print("ppp")
# Load the BARTScorer with a suitable model
bart_scorer = BARTScorer(device='cpu',checkpoint='facebook/bart-base')
print("loaded")
# Compute BARTScores
scores = bart_scorer.score(predictions, references, batch_size=4)
average_score = sum(scores) / len(scores)

print("Average BARTScore:", average_score)


In [ ]:
# Ask GPT with Biomedical Sentence Highlighting

def ask_gpt_highlight(question, context):
    prompt = (
        f"You are a biomedical expert. Given the abstract below, and the question:\n\n"
        f"--- ABSTRACT START ---\n{context}\n--- ABSTRACT END ---\n\n"
        f"Question: {question}\n\n"
        f"Please answer with one of: 'yes', 'no', or 'maybe'.\n"
        f"Then explain your reasoning in 1-2 sentences.\n"
        f"Finally, list 1-3 sentences from the abstract that best support your answer.\n\n"
        f"Format:\nAnswer: <yes/no/maybe>\nExplanation: <your explanation>\nSupporting Sentences: <copied sentences from abstract>\n\n"
    )

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )

    return response.choices[0].message.content


In [ ]:
results = []

for i, row in df.iterrows():
    print(f"Processing question {i+1}...")
    gpt_response = ask_gpt_highlight(row['QUESTION'], row['CONTEXT'])

    # Extract components using regex
    answer_match = re.search(r"(?i)Answer:\s*(yes|no|maybe)", gpt_response)
    explanation_match = re.search(r"(?i)Explanation:\s*(.+?)\n(?:Supporting Sentences|$)", gpt_response, re.DOTALL)
    support_match = re.search(r"(?i)Supporting Sentences:\s*(.+)", gpt_response, re.DOTALL)

    predicted = answer_match.group(1).lower() if answer_match else "unknown"
    explanation = explanation_match.group(1).strip() if explanation_match else ""
    support = support_match.group(1).strip() if support_match else ""

    results.append({
        'question': row['QUESTION'],
        'ground_truth': row['ANSWER'],
        'predicted_answer': predicted,
        'gpt_explanation': explanation,
        'supporting_sentences': support
    })

# Save and evaluate
results_df = pd.DataFrame(results)
results_df.to_csv("gpt4_pubmedqa_explained_highlighted.csv", index=False)

accuracy = (results_df['ground_truth'].str.lower() == results_df['predicted_answer']).mean()
print(f"\nAccuracy on {len(results_df)} examples: {accuracy*100:.2f}%")
print("Saved results to gpt4_pubmedqa_explained_highlighted.csv")


In [ ]:
#      Great choice! We can use SBERT (Sentence-BERT) to compute semantic similarity between:

# Sentences from the abstract, and

# The supporting sentences identified by GPT-4.

# This allows you to verify whether GPT-4 actually picked the most relevant evidence from the abstract — a strong step toward explainability and trust.

# ✅ Overview of the SBERT-based Cross-Check Process
# 🧩 Components:
# Break abstract into sentences (context_sents)

# Extract GPT's claimed supporting sentences (gpt_support)

# Compute SBERT embeddings for both

# Compute cosine similarity

# Find top N sentences closest to GPT's selections

# Report whether they match the claimed support

# ✅ Installation (Anaconda Terminal)
# bash
# Copy
# Edit
# pip install sentence-transformers scikit-learn
# ✅ Add This to Your Script
# 🔹 Step 1: SBERT Setup
# python
# Copy
# Edit
# from sentence_transformers import SentenceTransformer, util

# # Load SBERT model (biomed model available too)
# sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast and general
# 🔹 Step 2: Cross-Check Function
# python
# Copy
# Edit
# from sklearn.metrics.pairwise import cosine_similarity
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import sent_tokenize

# def cross_check_similarity(context, gpt_support):
#     # Split context and GPT supporting sentences
#     context_sents = sent_tokenize(context)
#     gpt_sents = sent_tokenize(gpt_support)

#     # Get embeddings
#     context_emb = sbert_model.encode(context_sents, convert_to_tensor=True)
#     gpt_emb = sbert_model.encode(gpt_sents, convert_to_tensor=True)

#     # For each GPT-support sentence, find top match from context
#     match_results = []
#     for i, gpt_vector in enumerate(gpt_emb):
#         cos_sim = util.pytorch_cos_sim(gpt_vector, context_emb)[0]
#         best_idx = cos_sim.argmax().item()
#         similarity_score = cos_sim[best_idx].item()
#         match_results.append({
#             "gpt_sentence": gpt_sents[i],
#             "matched_context_sentence": context_sents[best_idx],
#             "similarity_score": round(similarity_score, 3)
#         })

#     return match_results
# 🔹 Step 3: Run SBERT Cross-Check After GPT Response
# In your main loop (after getting GPT output):

# python
# Copy
# Edit
#     # SBERT cross-check
#     sbert_matches = cross_check_similarity(row['context'], support)
#     results.append({
#         'question': row['question'],
#         'ground_truth': row['answer'],
#         'predicted_answer': predicted,
#         'gpt_explanation': explanation,
#         'supporting_sentences': support,
#         'sbert_matched_sentences': sbert_matches
#     })
# If you're saving to a file, convert sbert_matched_sentences to string:

# python
# Copy
# Edit
# import json

# results_df['sbert_matched_sentences'] = results_df['sbert_matched_sentences'].apply(json.dumps)
# 📊 Example Output (Console or CSV)
# json
# Copy
# Edit
# [
#   {
#     "gpt_sentence": "Treatment X improved patient survival.",
#     "matched_context_sentence": "Patients treated with X had significantly better survival rates.",
#     "similarity_score": 0.88
#   }
# ]
# 🔍 Use Cases for This
# Identify hallucinated support (GPT picked a sentence not in the abstract).

# Quantify the factual alignment of explanation vs. source.

# Flag weak justifications (low similarity).

# Would you like to flag explanations under a similarity threshold (e.g. < 0.7) as unreliable?








# You said:
# yes


# ChatGPT said:








In [ ]:
for i, row in df.iterrows():
    print(f"Processing question {i+1}...")
    
    # Extract components using regex
    answer_match = re.search(r"(?i)Answer:\s*(yes|no|maybe)", gpt_response)
    explanation_match = re.search(r"(?i)Explanation:\s*(.+?)\n(?:Supporting Sentences|$)", gpt_response, re.DOTALL)
    support_match = re.search(r"(?i)Supporting Sentences:\s*(.+)", gpt_response, re.DOTALL)

    predicted = answer_match.group(1).lower() if answer_match else "unknown"
    explanation = explanation_match.group(1).strip() if explanation_match else ""
    support = support_match.group(1).strip() if support_match else ""

    # Ensure 'support' is a string if it is a list or other object
    if isinstance(support, list):
        support = " ".join(support)

    # SBERT cross-check
    sbert_matches = cross_check_similarity(row['context'], support)

    # Flag explanations with low similarity (< 0.7)
    flagged_sentences = [match for match in sbert_matches if match['similarity_score'] < 0.7]

    results.append({
        'question': row['question'],
        'ground_truth': row['answer'],
        'predicted_answer': predicted,
        'gpt_explanation': explanation,
        'supporting_sentences': support,
        'sbert_matched_sentences': sbert_matches,
        'flagged_sentences': flagged_sentences
    })

# Save and evaluate
results_df = pd.DataFrame(results)
results_df.to_csv("gpt4_pubmedqa_explained_highlighted_flagged.csv", index=False)

accuracy = (results_df['ground_truth'].str.lower() == results_df['predicted_answer']).mean()
print(f"\nAccuracy on {len(results_df)} examples: {accuracy*100:.2f}%")
print("Saved results to gpt4_pubmedqa_explained_highlighted_flagged.csv")


In [ ]:
##Few Shot prompting technique

In [ ]:
# Few-shot prompt examples
few_shot_examples = [
    {
        "question": "Does the study support the use of aspirin in preventing heart attacks?",
        "context": "A randomized study of 20,000 participants showed a 25% reduction in heart attacks among those who took aspirin regularly compared to placebo.",
        "answer": "Yes",
        "explanation": "The abstract clearly states a 25% reduction in heart attacks among aspirin users, indicating a positive effect."
    },
    {
        "question": "Is metformin effective in treating Alzheimer's disease?",
        "context": "Although metformin has shown promise in laboratory models, clinical trials in humans have not demonstrated significant cognitive improvement.",
        "answer": "No",
        "explanation": "While lab results were positive, the abstract confirms that human trials did not show cognitive benefit, so the answer is no."
    },
    {
        "question": "Does this paper conclude that the new vaccine is safe?",
        "context": "The phase 3 clinical trial reported minimal side effects and no serious adverse events among 15,000 participants.",
        "answer": "Yes",
        "explanation": "The trial reported minimal side effects and no serious adverse events, suggesting the vaccine is safe."
    }
]

In [ ]:
# Format few-shot prompt block
def format_few_shot_block(examples):
    prompt = ""
    for ex in examples:
        prompt += f"""Abstract:
{ex['context']}

Question: {ex['question']}
Answer: {ex['answer']}
Explanation: {ex['explanation']}

"""
    return prompt

In [ ]:
few_shot_block = format_few_shot_block(few_shot_examples)


In [ ]:
pip install --upgrade openai


In [109]:
# Build full prompt for each instance
def ask_gpt_few_shot(question, context):
    full_prompt = (
        "You are a biomedical research assistant helping answer clinical questions from abstracts.\n\n"
        "Below are some examples of question answering with explanations based on biomedical abstracts:\n\n"
        f"{few_shot_block}"
        f"Abstract:\n{context}\n\n"
        f"Question: {question}\n"
        f"Answer:"
    )


    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.0,
        max_tokens=120,
    )

    print(response.choices[0].message.content)


    return response.choices[0].message.content


In [73]:
# Run on one sample
sample = df.iloc[0]
response = ask_gpt_few_shot(sample['QUESTION'], sample['CONTEXT'])

Yes
Explanation: The abstract discusses a study that elucidates the role of mitochondrial dynamics during developmentally regulated PCD in the lace plant. The results suggest that mitochondrial dynamics are involved in the process of PCD in the lace plant, which is responsible for the formation of perforations in its leaves.


In [75]:
from datasets import load_dataset

# Load pubmedqa dataset (we'll use the 'pqa_labeled' subset)
dataset = load_dataset("pubmed_qa", "pqa_labeled")

# Select a smaller test subset for demonstration
examples = dataset['train'].select(range(10))  # You can increase this later


In [111]:
results = []

for i, row in df.iterrows():
    print(f"Processing question {i+1}...")
    gpt_response = ask_gpt_few_shot(row['QUESTION'], row['CONTEXT'])
    predicted = extract_answer(gpt_response)

    results.append({
        'question': row['QUESTION'],
        'ground_truth': row['ANSWER'],
        'predicted_answer': predicted,
        'gpt_explanation': gpt_response,
         'reference_explanation': row['CONTEXT']  # <-- add this line
    })


Processing question 1...
Yes
Explanation: The abstract discusses a study that elucidates the role of mitochondrial dynamics during developmentally regulated PCD in the lace plant. The results suggest that mitochondrial dynamics are involved in the process of PCD in the lace plant, which is responsible for the formation of perforations in its leaves.
Processing question 2...
Small
Explanation: The abstract mentions that the mean difference between Landolt C acuity (LR) and Snellen E acuity (SE) was 0.55 lines for the eyes with strabismus amblyopia, indicating only a small difference.
Processing question 3...
Yes
Explanation: The abstract discusses a case where infants became pale, hypotonic, still and unreactive during bathing, which is similar to symptoms of aquagenic urticaria. The infants also showed an increase in blood histamine levels after a trial bath, which is a common reaction in urticaria. Therefore, it can be inferred that the infants might have a pediatric form of water-ind

In [113]:
results_df = pd.DataFrame(results)
results_df.to_csv("gpt4_pubmedqa_results_few_shot.csv", index=False)

accuracy = (results_df['ground_truth'].str.lower() == results_df['predicted_answer']).mean()
print(f"\nAccuracy on {len(results_df)} examples: {accuracy*100:.2f}%")
print("Saved results to gpt4_pubmedqa_results_few_shot.csv")


Accuracy on 10 examples: 60.00%
Saved results to gpt4_pubmedqa_results_few_shot.csv


In [115]:
#EXACT MATCH

correct = 0
total = len(results)

for r in results:
    if r['predicted_answer'].strip().lower() == r['ground_truth'].strip().lower():
        correct += 1

accuracy = correct / total * 100
print(f"Exact Match Accuracy: {accuracy:.2f}%")



Exact Match Accuracy: 60.00%


In [117]:
from bert_score import score

# Extract lists of explanations
gpt_explanations = [r['gpt_explanation'] for r in results]
reference_explanations = [r['reference_explanation'] for r in results]

P, R, F1 = score(gpt_explanations, reference_explanations, lang="en", model_type="bert-base-uncased")
print(f"BERTScore F1: {F1.mean().item():.4f}")


BERTScore F1: 0.3216


In [119]:
from bert_score import score as bert_score
# from bart_score import BARTScorer

# Extract lists of predictions and references
predictions = [r['gpt_explanation'] for r in results]
references = [r['reference_explanation'] for r in results]  # You need to have this in your data

# -------- BERTScore --------
# P, R, F1 = bert_score(predictions, references, lang="en", model_type="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", verbose=True)
from bert_score import score

P, R, F1 = score(
    predictions,
    references,
    lang="en",  # keep lang as 'en'
    model_type="bert-base-uncased",  # Use this instead
    verbose=True
)

# Average BERTScore F1
bert_f1_avg = F1.mean().item()
print(f"Average BERTScore F1: {bert_f1_avg:.4f}")

# -------- BARTScore --------
# bart_scorer = BARTScorer(device='cuda' if torch.cuda.is_available() else 'cpu', checkpoint='facebook/bart-large-cnn')

# Compute scores
# bart_scores = bart_scorer.score(predictions, references, batch_size=4)
# bart_avg = sum(bart_scores) / len(bart_scores)
# print(f"Average BARTScore: {bart_avg:.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.47 seconds, 20.26 sentences/sec
Average BERTScore F1: 0.3216


In [85]:
predictions = [str(r['gpt_explanation']) for r in results if r['gpt_explanation'] is not None]
references = [str(r['reference_explanation']) for r in results if r['reference_explanation'] is not None]



In [87]:
from bart_score import BARTScorer
print("ppp")
# Load the BARTScorer with a suitable model
bart_scorer = BARTScorer(device='cpu',checkpoint='facebook/bart-base')
print("loaded")
# Compute BARTScores
scores = bart_scorer.score(predictions, references, batch_size=4)
average_score = sum(scores) / len(scores)

print("Average BARTScore:", average_score)


ppp
loaded
Average BARTScore: -7.9501815392420845
